In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load data
sector_data = pd.read_csv("sector_data.csv")
stock_data = pd.read_csv("stock_prices.csv")

# Compute sector-wise averages
sector_pe_avg = stock_data.groupby("Sector")[["Current P/E Ratio", "Historical Avg P/E Ratio"]].mean()
sector_networth_avg = stock_data.groupby("Sector")[["Net Worth ($B)"]].mean()

# Merge with sector data
sector_data = sector_data.merge(sector_pe_avg, on="Sector")
sector_data = sector_data.merge(sector_networth_avg, on="Sector", suffixes=("", "_avg"))

# Check the columns after merging
print("Columns after merge:", sector_data.columns)

# Select stocks based on criteria
filtered_stocks = []
for _, row in stock_data.iterrows():
    sector = row["Sector"]
    
    # Extract sector-wise values
    sector_row = sector_data.loc[sector_data["Sector"] == sector]
    
    # Ensure the sector exists in sector_data before accessing values
    if not sector_row.empty:
        pe_diff = abs(row["Current P/E Ratio"] - row["Historical Avg P/E Ratio"])
        sector_pe_diff = abs(sector_row["Current P/E Ratio"].values[0] - sector_row["Historical Avg P/E Ratio"].values[0])
        
        # Ensure the correct column name is being accessed
        if "Net Worth ($B)" in sector_row.columns:
            min_networth = 0.6 * sector_row["Net Worth ($B)"].values[0]
        else:
            print(f"Column 'Net Worth ($B)' is missing in sector_data for sector {sector}")
            continue
        
        if pe_diff < sector_pe_diff and row["Net Worth ($B)"] > min_networth:
            filtered_stocks.append(row)

# Convert list to DataFrame
filtered_stocks_df = pd.DataFrame(filtered_stocks)

# If no stocks pass the filter, handle gracefully
if filtered_stocks_df.empty:
    print("No stocks meet the filtering criteria.")
else:
    # Assign weights based on sector Avg Lowest Price
    if "Avg Lowest" in sector_data.columns:
        sector_lowest_avg = sector_data.set_index("Sector")["Avg Lowest"]
        sector_weights = sector_lowest_avg / sector_lowest_avg.sum()
        filtered_stocks_df["Sector Weight"] = filtered_stocks_df["Sector"].map(sector_weights)
    else:
        print("Column 'Avg Lowest' not found in sector_data. Skipping weight assignment.")
        filtered_stocks_df["Sector Weight"] = np.nan  # Placeholder if missing

    # Prepare data for regression
    if "Sector Weight" in filtered_stocks_df.columns and "Last Price ($)" in filtered_stocks_df.columns:
        X = filtered_stocks_df[["Sector Weight"]].dropna()  # Drop NaN values if any
        y = filtered_stocks_df.loc[X.index, "Last Price ($)"]  # Ensure y matches X indices

        if not X.empty and not y.empty:
            # Train Linear Regression Model
            model = LinearRegression()
            model.fit(X, y)

            # Predict investment allocation
            predictions = model.predict(X)
            filtered_stocks_df.loc[X.index, "Investment Weight"] = predictions / predictions.sum()

            # Display selected stocks with investment allocation
            print(filtered_stocks_df[["Sector", "Stock", "Last Price ($)", "Investment Weight"]])
        else:
            print("Not enough data for regression model.")
    else:
        print("Required columns for regression are missing.")


Columns after merge: Index(['Sector', 'Avg Last Price', 'Avg Open Price', 'Avg Highest',
       'Avg Lowest', 'Current P/E Ratio', 'Historical Avg P/E Ratio',
       'Net Worth ($B)'],
      dtype='object')
                  Sector  Stock  Last Price ($)  Investment Weight
1             Technology   MSFT          365.33           0.073515
2             Technology  GOOGL          148.47           0.073515
4             Technology   META          508.45           0.073515
5   Banking & Financials    JPM          213.63           0.052716
6   Banking & Financials    BAC           34.92           0.052716
8   Banking & Financials      C           58.32           0.052716
10        Consumer Goods     PG          172.77           0.037610
11        Consumer Goods     KO           72.79           0.037610
12        Consumer Goods    PEP          154.25           0.037610
20                Energy    XOM          108.37           0.036255
21                Energy    CVX          149.60         

In [3]:
from sklearn.metrics import mean_squared_error, r2_score

# Evaluate the model
predictions = model.predict(X)
mse = mean_squared_error(y, predictions)
r2 = r2_score(y, predictions)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 13078.670309268282
R-squared: 0.1614749286892293
